In [1]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go


In [ ]:
d_costdata = pd.read_csv('costs_parameters_nbnlscenariokm.csv')
c_costdata = pd.read_csv('costs_parameters_10pincrease.csv')

scenarios = [d_costdata, c_costdata]

In [ ]:
#Overview of the Costs for each Group
costs_groupdata = {'Group': ["Building and Installations", "Energy Production", "Storage and Conversion", "Carriers", "Infrastructure", "CCUS", "CO2 Emsissions", "Captured Biogenic CO2"],
                   'Default Cost In Billion Euros': [d_costdata[d_costdata['Group']=="costs_building_and_installations"]['Total costs (eur)'].sum(),
                                             d_costdata[d_costdata['Group']=="costs_production"]['Total costs (eur)'].sum(),
                                             d_costdata[d_costdata['Group']=="costs_storage_and_conversion"]['Total costs (eur)'].sum(),
                                             d_costdata[d_costdata['Group']=="costs_carriers"]['Total costs (eur)'].sum(),
                                             d_costdata[d_costdata['Group']=="costs_infrastructure"]['Total costs (eur)'].sum(),
                                             d_costdata[d_costdata['Group']=="costs_co2"]['Total costs (eur)'].sum(),
                                             d_costdata['CO2 emission costs (eur)'].sum(),
                                             d_costdata['Captured biogenic CO2 costs (eur)'].sum()
                                            ],
                   'Current Cost In Billion Euros': [c_costdata[c_costdata['Group']=="costs_building_and_installations"]['Total costs (eur)'].sum(),
                                             c_costdata[c_costdata['Group']=="costs_production"]['Total costs (eur)'].sum(),
                                             c_costdata[c_costdata['Group']=="costs_storage_and_conversion"]['Total costs (eur)'].sum(),
                                             c_costdata[c_costdata['Group']=="costs_carriers"]['Total costs (eur)'].sum(),
                                             c_costdata[c_costdata['Group']=="costs_infrastructure"]['Total costs (eur)'].sum(),
                                             c_costdata[c_costdata['Group']=="costs_co2"]['Total costs (eur)'].sum(),
                                             c_costdata['CO2 emission costs (eur)'].sum(),
                                             c_costdata['Captured biogenic CO2 costs (eur)'].sum()
                                            ]
                   }
costs_group = pd.DataFrame(costs_groupdata)

total_df = pd.DataFrame(
    {'Group': ['Total'], 
     'Default Cost In Billion Euros': [(costs_group['Default Cost In Billion Euros'].sum()/1000000000).round(3)], 
     'Current Cost In Billion Euros': [(costs_group['Current Cost In Billion Euros'].sum()/1000000000).round(3)]
                         }
                         )
costs_group['Default Cost In Billion Euros'] = (costs_group['Default Cost In Billion Euros']/1000000000).round(3)
costs_group['Current Cost In Billion Euros'] = (costs_group['Current Cost In Billion Euros']/1000000000).round(3)
costs_group = pd.concat([costs_group, total_df], ignore_index=True)
costs_group['Change (billion euros)'] = (costs_group['Current Cost In Billion Euros'] - costs_group['Default Cost In Billion Euros']).round(3)
costs_group['Change (%)'] = ((costs_group['Change (billion euros)']/costs_group['Default Cost In Billion Euros'])*100).round(2)

costs_group

,Group,Default Cost In Billion Euros,Current Cost In Billion Euros,Change (billion euros),Change (%)
0,Building and Installations,13.256,13.121,-0.135,-1.02
1,Energy Production,13.539,13.595,0.056,0.41
2,Storage and Conversion,18.448,18.446,-0.002,-0.01
3,Carriers,31.015,31.053,0.038,0.12
4,Infrastructure,20.739,21.167,0.428,2.06
5,CCUS,0.949,0.949,0.000,0.00
6,CO2 Emsissions,0.173,0.173,0.000,0.00
7,Captured Biogenic CO2,-1.963,-1.963,0.000,-0.00
8,Total,96.157,96.543,0.386,0.40


In [ ]:
#Buildings and Installations Cost Breakdown into Subgroups
cbi_subgroups = d_costdata[d_costdata['Group'] == "costs_building_and_installations"]
cbi_subgroups = cbi_subgroups.groupby("Subgroup")['Total costs (eur)'].sum()

cbi_total = (cbi_subgroups.sum()/1000000000).round(3)
total_df = pd.DataFrame({'Subgroup': ['Total'], 'Cost In Billion Euros': [cbi_total]})

cbi_subgroupsdata = {'Subgroup': cbi_subgroups.index.tolist(),
                     'Cost In Billion Euros': cbi_subgroups.values.tolist()
                    }
cbi_subgroups = pd.DataFrame(cbi_subgroupsdata)

cbi_subgroups['Cost In Billion Euros'] = (cbi_subgroups['Cost In Billion Euros']/1000000000).round(3)
cbi_subgroups = pd.concat([cbi_subgroups, total_df], ignore_index=True)
cbi_subgroups


In [189]:
for scenario in scenarios:
    inf_costs = d_costdata[d_costdata['Group']=="costs_infrastructure"]

    hydrogen_carriers = scenario[scenario['Subgroup']=="hydrogen"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="liquid_hydrogen"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="lohc"]['Total costs (eur)'].sum()
    natural_gas =       scenario[scenario['Subgroup']=="lng"]['Total costs (eur)'].sum() + scenario[scenario['Subgroup']=="network_gas"]['Total costs (eur)'].sum()
    electricity =       scenario[scenario['Subgroup']=="electricity"]['Total costs (eur)'].sum()
    ammonia =           scenario[scenario['Subgroup']=="ammonia"]['Total costs (eur)'].sum()
    oil_p =             scenario[scenario['Subgroup']=="diesel"]['Total costs (eur)'].sum()
    heat =              scenario[scenario['Subgroup']=="heat"]['Total costs (eur)'].sum()

    if scenario.equals(d_costdata):
        inf_subgroupsdata = {'Subgroup': ["Natural Gas", "Hydrogen Carriers", "Electricity", "Heat", "Ammonia", "Oil and Products"],
                        'Default Cost In Billion Euros': [natural_gas, hydrogen_carriers, electricity, heat, ammonia, oil_p]
                        }
        inf_subgroups = pd.DataFrame(inf_subgroupsdata)
    else:
        inf_subgroupsdata = {'Subgroup': ["Natural Gas", "Hydrogen Carriers", "Electricity", "Heat", "Ammonia", "Oil and Products"],
                        'Current Cost In Billion Euros': [natural_gas, hydrogen_carriers, electricity, heat, ammonia, oil_p]
                        }
        inf_subgroups_current = pd.DataFrame(inf_subgroupsdata)
        inf_subgroups = pd.concat([inf_subgroups, inf_subgroups_current], axis=1)

print(inf_subgroups)

# inf_subgroups = pd.DataFrame(inf_subgroupsdata)
# inf_total = (inf_subgroups['Cost In Billion Euros'].sum()/1000000000).round(3)
# total_df['Cost In Billion Euros'] = inf_total
# inf_subgroups['Cost In Billion Euros'] = (inf_subgroups['Cost In Billion Euros']/1000000000).round(3)
# inf_subgroups = pd.concat([inf_subgroups, total_df], ignore_index=True)

# inf_subgroups

            Subgroup  Default Cost In Billion Euros
0        Natural Gas                   1.493751e+09
1  Hydrogen Carriers                  -1.804338e+09
2        Electricity                   1.571198e+10
3               Heat                   1.506166e+09
4            Ammonia                   1.298214e+10
5   Oil and Products                   0.000000e+00


Subcategories hydrogen, liquid_hydrogen, and lohc are combined into one in gquerry under hydrogen_carriers
Subcategories lng and network gas are combined into one category in gquerry under natural_gas
Heat is made up of HT, MT and LT heat
Ammonia is made up of capital and operating expenses
Subcategory diesel appears as oil_and_products in gquery and is breaken down into capital and operating expenses
